#  DATA INTERPRETER
This Jupyter notebook is used to visualize and interpret the processed data. In other words, we will conduct the analysis on the sample distribution of car sales in the Czech republic through a bar chart and a heatmap. Moreover, the analysis will go on with a Machine Learning model which first splits the data into sets of data for training and testing, then predicts prices for cars for which the user can insert input details ranging from year to mileage to location (region this time). 

But we first need to continue processing the data!

#  Import of packages
Numpy, Plotly, pandas and json packages will be used in the first part of this Jupyter notebook in order to combine and process the datasets. For visualisation Plotly will be used.

In [1]:
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

In [3]:
__name__ == "__main__"

True

In [4]:
df1 = pd.read_csv('merged_car_data.csv', ',')
df2 = pd.read_csv('data_okresy.csv', ',')

In [4]:
big_data = pd.concat([df1, df2], axis = 1)
big_data = big_data.drop(columns = 'Unnamed: 0')

Load the geojson file containing data on the geographical coordinates of the Czech districts.

In [5]:
czech_regions = json.load(open("distictsCzechiaLow.json", encoding = "UTF-8"))
#czech_regions['features'][0]['properties']

Now all we need to do is matching coordinates from the json file with the districts in our merged dataset.

In [6]:
state_id_map = {}
for feature in czech_regions["features"]:
    feature["id"] = feature["properties"]["id"]
    state_id_map[feature["properties"]["name"]] = feature["id"]

In [7]:
state_id_map
mapping = pd.DataFrame([state_id_map]).T
mapping.columns = ['id']
mapping = mapping.reset_index().rename(columns = {'index': 'All Okresy'})
mapping.columns

Index(['All Okresy', 'id'], dtype='object')

In [8]:
bigDataGrouped = big_data.groupby(['Okres']).size().reset_index(name='counts')
new = bigDataGrouped["Okres"].str.split("\n ", n = 1, expand = True)
bigDataGrouped["Okres"] = new[0]

Note that the below script makes a slight change in names of some districts in our initial dataset for perfect matching. Using the method .groupby(), we are able to see how many cars there are in each Czech district. For the districts that are not available in this dataset but available in the geojson file, we will aoutomatically assign 0 amount of cars with the codes below.

In [9]:
bigFinal = bigDataGrouped.groupby(['Okres'], as_index=False).sum()
bigFinal.Okres.replace({
    'Hlavní město Praha':'Praha',
    'Rychnov nad Kněžnou':'Rychnov n.K.',
    'Ústí nad Orlicí':'Ústí n.O.',
    'Žďár nad Sázavou':'Žďár n.S.'
    },
    inplace = True
)
bigFinal['id'] = bigFinal['Okres'].apply(lambda x: state_id_map[x])
#bigFinal

In [10]:
bigFinal
mapping.id
allFinal = bigFinal.merge(mapping, on = 'id', how = 'right').replace(np.nan, 0).drop(columns = 'Okres')
#allFinal

We have put two options for the interactive map; the second is more vivid, in our opinions :)

P.S. Please drag it to Europe in the monitor, so you can view territory of the Czech Republic.

In [252]:
#fig = px.choropleth(allFinal, locations = 'id', geojson = czech_regions, color = 'counts', hover_data=["counts"])
#fig.update_geos(fitbounds="locations", visible=False)
#fig.show()

In [11]:
#The map will be opened in your browser for not putting too much "pressure" on the Jupyter Notebook

fig2 = px.choropleth_mapbox(
    allFinal,
    locations="id",
    geojson=czech_regions,
    color="counts",
    hover_name="All Okresy",
    hover_data=["counts"],
    title="Car Sales per District",
    mapbox_style="carto-positron",
    center={"lat": 50, "lon": 15},
    zoom=6,
    opacity=0.7,
)
fig2.show()

#  Bar Chart

Now we will import necessary packages, most importantly from bokeh, in order to construct an illuminating bar chart that shows the distribution of 53 different car brands.

In [5]:
from matplotlib import pyplot as plt
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, curdoc
from bokeh.palettes import plasma, turbo
from bokeh.layouts import row, column
from collections import Counter

car_brands = df1['Name'].str.split(" ", n = 1, expand = True)[0]
brand_names = Counter(car_brands).keys()
brand_frequency = Counter(car_brands).values()
brand_names = list(brand_names)
brand_frequency = list(brand_frequency)
sorted_brands = sorted(brand_names, key=lambda x: brand_frequency[brand_names.index(x)])


p2 = figure(title = "Distribution of Brands in car sales in Czechia (simple plot)",
            x_range = sorted_brands,
            plot_height=660,
            plot_width = 980,
            toolbar_location = None)

p2.vbar(x = brand_names, 
        top= brand_frequency, 
        width = 0.9,
        color = plasma(53))

#Getting rid of gridlines of x axis
p2.xgrid.grid_line_color = None

#Starting the y axis from 500 rather than 0
p2.y_range.start = 0

#Rotating labels such that they do not overlap
p2.xaxis.major_label_orientation = 1

#Adding y axis label
p2.yaxis.axis_label = 'Number of sales per one brand'

#Display the bar chart
show(p2)

#  Next part with Machine Learning model

We will know import necessary stuff from the Scikit-learn library of Python in order to create a nice Linear Regression. But we first need to clean the data as the numbers such as mileages and prices are still string type of parameters in the dataset.

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from tqdm import tqdm
import unicodedata
import unidecode

We will split the price column from initial dataset as we are going to predict prices for cars. Also, we will trim the Name column for creating a nicer column called Brand, which is way more useful for prediction. We will also drop the Obec column as we will use Regions for prediction.

In [11]:
df1['Brand'] = df1['Name'].str.split(" ", n = 1, expand = True)[0]
df1.Year = df1.Year.apply(lambda x: pd.to_numeric(x))
df1 = df1.dropna()
y = df1.Price
x = df1.drop(['Price', 'Name', 'Obec'], axis=1)
x.Mileage = x.Mileage.str.split("km", n = 1, expand = True)[0]
y = y.str.split('Kč', n = 1, expand = True)[0]
x = x.reset_index().drop(columns = ['index'])
y = y.reset_index().drop(columns = 'index')
y.columns = ['Price']

Below we are cleaning the data about Price and Mileage by converting them into integers, just as they still carry some string values that we do not like, despite we used "utf-8" in scraping.

In [12]:
intPrice = []
intMileage = []
for i in y.Price:
    intPrice.append(int(i.replace(u'\xa0', u'')))
for j in x.Mileage:
    intMileage.append(int(j.replace(u'\xa0', u'')))

Now for the column Engine, we can simplify things by creating two categorical variables "Benzin" and "Nafta". Thus, we do not really need to complicate things with dozens of Engine types with numerical Kilowatts. Also, since the column Body contains values such as "Suv", "suV", and "SUV", we will convert all values to lowercase versions.

In [13]:
conditions = [
    x['Engine'].str.contains('Benzín'),
    x['Engine'].str.contains('Nafta')]
    
values = ['Benzín', 'Nafta']

x['Engine'] = np.select(conditions, values)
x['Body'] = x.Body.str.lower()

Now we can finish the data processing and create the dummy variables and then split the data into training and testing sets.

In [37]:
x.Mileage = intMileage
y.Price = intPrice
x.Year = x.Year.astype(int)
dummy = pd.get_dummies(x, drop_first = True)
pd.set_option('display.max_columns', None)
dummyCols = dummy.columns.tolist()
X = dummy
X = X.drop(columns = ['Location_NEJNIŽŠÍ MOŽNÉ SPLÁTKY NA ÚVĚRY!!!', "Unnamed: 0"])

In [38]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.33, random_state=0)

In [39]:
LR = LinearRegression()
LR.fit(X_train,y_train)

LinearRegression()

In [40]:
y_prediction =  LR.predict(X_test)
y_prediction

array([[ 35115.25198611],
       [181304.26675663],
       [267545.59954252],
       ...,
       [167316.51033074],
       [212849.76193966],
       [ 60860.64780837]])

Below we are importing additional packages for CrossValidation and showing the R^2 value.

In [22]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [107]:
score=r2_score(y_test,y_prediction)
print(f'r2 score is {score}')

r2 score is 0.5225459907781258


As we can see from the above value, the model is not at its best, but not the worst either.

In [31]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(LR, X, y, scoring='r2', cv=cv, n_jobs=-1)
scores

array([ 0.51672341,  0.56509751,  0.46571261,  0.57356568,  0.55438765,
        0.44193563, -2.33253819,  0.58201809,  0.45167843,  0.49158461])

Now let us define a function which will help the users to predict price of cars of their own interest by inserting 7 input values, ranging from year to mileage to body to brand. Note that for prediction we create another dataframe which is then put into the ML model. Thus we first initialize all values with zeros, then the function converts input values to prediction values itself. Mind that, for a body type, for instance, if the user inserts input as "SUV", the function assigns the value 1 to the dummy variable "Body_suv", while others still have 0. User can convert all assigned values to zeros using the single line of code after the function.

In [86]:
allCols = X.columns.tolist()
Values = np.zeros(len(allCols))
carPredict = pd.DataFrame(Values, allCols).T 

As we know, in a simple regression model, if we have K categorical variables, we will use K-1 dummy variables within the model (except for some models such as Decision Trees). Thus, the model below drops some variables and that is why we include if-else statements below in order to make sure that the model can read whenever putting an input of a dropped variable into the model. In other words, if the user inserts brand "Alfa", which is dropped from the dummy variables, the model will automatically assign zeros to the rest of the dummy variables.

In [106]:
def addPredictingParameters(year, mileage, engine, gearbox, body, location, brand):   
    carPredict['Year'][0] = year
    carPredict['Mileage'][0] = mileage
    
    if 'Engine_' + str(engine) in allCols:
        carPredict['Engine_' + str(engine)][0] = 1
    else:
        contains_engine = [col for col in carPredict.columns if 'Engine_' in col]
        carPredict[contains_engine] = 0
    
    if 'Gearbox_' + str(gearbox) in allCols:
        carPredict['Gearbox_' + str(gearbox)][0] = 1
    else:
        contains_gearbox = [col for col in carPredict.columns if 'Gearbox_' in col]
        carPredict[contains_gearbox] = 0
        
    if 'Body_' + str(body) in allCols:
        carPredict['Body_' + str(body)][0] = 1
    else:
        contains_body = [col for col in carPredict.columns if 'Body_' in col]
        carPredict[contains_body] = 0
        
    if 'Location_' + str(location) in allCols:
        carPredict['Location_' + str(location)][0] = 1
    else:
        contains_location = [col for col in carPredict.columns if 'Location_' in col]
        carPredict[contains_location] = 0
        
    if 'Brand_' + str(brand) in allCols:
        carPredict['Brand_' + str(brand)][0] = 1
    else:
        contains_brand = [col for col in carPredict.columns if 'Brand_' in col]
        carPredict[contains_brand] = 0        
        
        
addPredictingParameters(2006, 220000, 'Benzín', 'Automatická', 'sedan', 'Praha', 'Mercedes-Benz')
print(LR.predict(carPredict))
carPredict = pd.DataFrame(pd.Series(0, index=carPredict.columns)).T

[[449540.42324271]]


In [2]:
#carPredict

#  Basic Interpretations

The visualization derived from the interactive map suggests that in a sample of 6000 car, most of them (or roughly 709) are available in Prague (Praha). Furthermore, there comes other prominent cities such as Brno, Plzen, Karlovy-vary, and Ceske Budejovice. Actually, this makes sense since those cities are concentrated with more people who naturally perform higher number of sales.

When it comes to the bar chart depicting brand distribution within our sample, the data show that Skoda is the most frequently represented car brand with a number of 990 cars. Skoda is then followed by other famous brands like Volkswagen, Ford, Renault, Mercedes, Audi and so on. The least represented brand was Infinity and some other US-based firms. The results give us a bit clearer insight about geographical and sosioeconomic tastes of people in the Czech Republic.

Lastly, speaking of the pricing model, it returns higher prices for newer cars, which is quite intuitive and thus in line with reality. Also, cars that are in sale in bigger cities are more expensive. Likewise, cars with the automatic gearbox are following the same pattern too. Last but not least, cars with a great record of mileage are less expensive as expected. 

Note that some of this results cannot be interpreted for whole car industry and market in the Czech Republic. It is nothing but a mere interpretation of ours derived from a sample of 6000 cars from the given website.